https://github.com/explosion/spaCy/blob/master/LICENSE

Using the parser to recognise your own semantics

spaCy's parser component can be used to trained to predict any type of tree
structure over your input text. You can also predict trees over whole documents
or chat logs, with connections between the sentence-roots used to annotate
discourse structure. In this example, we'll build a message parser for a common
"chat intent": finding local businesses. Our message semantics will have the
following types of relations: ROOT, PLACE, QUALITY, ATTRIBUTE, TIME, LOCATION.

- "show me the best hotel in berlin"
- ('show', 'ROOT', 'show')
- ('best', 'QUALITY', 'hotel') --> hotel with QUALITY best
- ('hotel', 'PLACE', 'show') --> show PLACE hotel
- ('berlin', 'LOCATION', 'hotel') --> hotel with LOCATION berlin

In [1]:
import random
import spacy
from pathlib import Path

In [2]:
# training data: texts, heads and dependency labels
# for no relation, we simply chose an arbitrary dependency label, e.g. '-'
TRAIN_DATA = [
    ("find a cafe with great wifi", {
        'heads': [0, 2, 0, 5, 5, 2],  # index of token head
        'deps': ['ROOT', '-', 'PLACE', '-', 'QUALITY', 'ATTRIBUTE']
    }),
    ("find a hotel near the beach", {
        'heads': [0, 2, 0, 5, 5, 2],
        'deps': ['ROOT', '-', 'PLACE', 'QUALITY', '-', 'ATTRIBUTE']
    }),
    ("find me the closest gym that's open late", {
        'heads': [0, 0, 4, 4, 0, 6, 4, 6, 6],
        'deps': ['ROOT', '-', '-', 'QUALITY', 'PLACE', '-', '-', 'ATTRIBUTE', 'TIME']
    }),
    ("show me the cheapest store that sells flowers", {
        'heads': [0, 0, 4, 4, 0, 4, 4, 4],  # attach "flowers" to store!
        'deps': ['ROOT', '-', '-', 'QUALITY', 'PLACE', '-', '-', 'PRODUCT']
    }),
    ("find a nice restaurant in london", {
        'heads': [0, 3, 3, 0, 3, 3],
        'deps': ['ROOT', '-', 'QUALITY', 'PLACE', '-', 'LOCATION']
    }),
    ("show me the coolest hostel in berlin", {
        'heads': [0, 0, 4, 4, 0, 4, 4],
        'deps': ['ROOT', '-', '-', 'QUALITY', 'PLACE', '-', 'LOCATION']
    }),
    ("find a good italian restaurant near work", {
        'heads': [0, 4, 4, 4, 0, 4, 5],
        'deps': ['ROOT', '-', 'QUALITY', 'ATTRIBUTE', 'PLACE', 'ATTRIBUTE', 'LOCATION']
    })
]

In [3]:
nlp = spacy.blank('en')

In [4]:
parser = nlp.create_pipe('parser')
nlp.add_pipe(parser, first=True)

In [5]:
for text, annotations in TRAIN_DATA:
    for dep in annotations.get('deps', []):
        parser.add_label(dep)

In [6]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'parser']
with nlp.disable_pipes(*other_pipes):  # only train parser
    optimizer = nlp.begin_training()
    for itn in range(5):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in TRAIN_DATA:
            nlp.update([text], [annotations], sgd=optimizer, losses=losses)
        print(losses)

{'parser': 47.59957589333203}
{'parser': 36.84128908866564}
{'parser': 22.815627198833546}
{'parser': 21.863796436639667}
{'parser': 7.818227429146986}


In [7]:
def test_model(nlp):
    texts = ["find a hotel with good wifi",
             "find me the cheapest gym near work",
             "show me the best hotel in berlin"]
    docs = nlp.pipe(texts)
    for doc in docs:
        print(doc.text)
        print([(t.text, t.dep_, t.head.text) for t in doc if t.dep_ != '-'])

In [8]:
test_model(nlp)

find a hotel with good wifi
[('find', 'ROOT', 'find'), ('hotel', 'PLACE', 'find'), ('good', 'QUALITY', 'wifi'), ('wifi', 'ATTRIBUTE', 'hotel')]
find me the cheapest gym near work
[('find', 'ROOT', 'find'), ('cheapest', 'QUALITY', 'gym'), ('gym', 'PLACE', 'find'), ('near', 'ATTRIBUTE', 'gym'), ('work', 'TIME', 'near')]
show me the best hotel in berlin
[('show', 'ROOT', 'show'), ('best', 'QUALITY', 'hotel'), ('hotel', 'PLACE', 'show'), ('berlin', 'LOCATION', 'hotel')]
